<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2020.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 3: Morphology</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday December 10, 2020</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Friday December 18, 2020</span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weigth</b>: 10% of the overall grade</p>
    <p style="margin:4px;"><b>Remote help</b>: Monday December 14 and Thursday December 17, 12h00-13h00, on Zoom (see Moodle for link)</p>    
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 4.3</p> 
</div>

### Student Name: Filip Slezak
### SCIPER: 286557

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [3]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

SCIPER: 286557


## Imports
In this first cell we import the required Python libraries:
* [`matplotlib.pyplot`](https://matplotlib.org), to display images
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays
* [`cv2`](https://docs.opencv.org/master/), for image processing in Python

We will then load the `IPLabViewer` class (either see the complete documentation [here](https://github.com/Biomedical-Imaging-Group/IPLabImageViewer/wiki/Python-IPLabViewer()-Class), run the Python command `help(viewer)` after loading the class, or refer to [Lab 0: Introduction](./Introductory.ipynb)).

Finally, we load the images you will use in the exercise to test your algorithms. 

In [4]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import cv2 as cv

# Import IPLabsImage() Class
import sys  
sys.path.insert(0, 'lib')
from iplabs import IPLabViewer as viewer

# Load images to be used in this lab 
plate        = cv.imread('images/plate.tif',               cv.IMREAD_UNCHANGED)
butterfly    = cv.imread('images/butterfly-graylevel.tif', cv.IMREAD_UNCHANGED)
test_contact = cv.imread('images/test-contact.tif',        cv.IMREAD_UNCHANGED)
test_scratch = cv.imread('images/test-scratch.tif',        cv.IMREAD_UNCHANGED)

In the following cell we import the JavaScript `IPLabImageAccess` class, created specifically for this course, which facilitates the creation and modification of images.

In [5]:
%use javascript
// import IPLabImageAccess.js as Image
var Image = require('./lib/IPLabImageAccess.js')

# Morphology (17 points)

In this laboratory you will learn 1) how to implement several morphological filters in a low-level language (like JavaScript), and 2) how to apply them in image processing applications using OpenCV. We will focus mainly on 2D graylevel morphology, but keep in mind that the same operations can easily be adapted to color images by treating each color channel as an independent graylevel image.

## <a class="anchor"></a> Index
1. [Structuring Elements](#-1.-Structuring-elements-(2-points)) (**2 points**)
    1. [Square](#-1.A.-Square-structuring-element)
    2. [Cross](#-1.B.-Cross-structuring-element)
    3. [Disc](#-1.C.-Disc-structuring-element)
2. [Debugging](#-2.-Debugging-(1-point)) (**1 point**)
3. [Morphological filters](#-3.-Morphological-filters-(9-points)) (**9 points**)
    1. [Erosion](#-3.A.-Erosion)
    2. [Dilation](#-3.B.-Dilation)
    3. [Median](#-3.C.-Median-filter)
    4. [Opening](#-3.D.-Opening)
    5. [Closing](#-3.E.-Closing)
    6. [Gradient](#-3.F.-Gradient-filter)
    7. [Top-hat](#-3.G.-Top-hat-filter)
    8. [Bottom-hat](#-3.H.-Bottom-hat-filter)
    9. [Understanding morphological filters](#-3.I.-Understanding-morphological-filters)
4. [Direct applications](#-4.-Direct-applications-(2-points)) (**2 points**)
    1. [Disconnecting round objects](#-4.A.-Disconnecting-round-objects)
    2. [Detecting horizontal lines](#-4.B.-Detecting-horizontal-lines)
5. [Combining morphological filters](#-5.-Combining-morphological-filters-(1-point)) (**1 point**)
6. [Lantuéjoul's skeleton](#-6.-Lantuéjoul's-skeleton-(2-points)) (**2 points**)
    1. [Classic Lantuéjoul's algorithm](#-6.A.-Classic-Lantuéjoul's-algorithm)
        1. [Implementing skeletonize](#-6.A.a.-Implementing-skeletonize)
        2. [Testing skeletonize](#-6.A.b.-Testing-skeletonize)
    2. [Pruning and post-processing](#-6.B.-Pruning-and-post-processing)
        1. [Implementing pruning](#-6.B.a.-Implementing-pruning)
        2. [Testing skeletonize and prune](#-6.B.b.-Testing-skeletonize-and-prune)
7. [Cartoonize your picture!](#-7.-Cartoonize-your-picture!)

<div class=" alert alert-danger">
    
<b>Important:</b> Each cell that contains code begins with `%use sos` or `%use javascript`. This indicates if the code in this specific cell should be written in Python or JavaScript. Do not change or remove any lines of code that begin with an $\%$. They need to be on the first line of each cell!
    
</div>

# <a class="anchor"></a> 1. Structuring elements (2 points)
[Back to index](#-Index)

As you have seen in the course, a structuring element (often abbreviated with `strel` in the code) can be discribed by a binary image consisting of a foreground and a background, which are usually represented with `true` (or $1$) and `false` (or $0$), respectively. They are used in morphological filters such as dilation, erosion, opening, closing and all others that are derived from those. The purpose of the structuring element is to define the shape and size used by a morphological filter.

**Your task in this section is to implement 2 different structuring elements, namely a cross and a disc.**

**For 1 point each**, implement the functions `cross(n)` and `disc(n)` to build the shapes of the structuring elements of size $n \times n$. The function `square(n)` is provided and serves as a reference. The "disc" shape contains elements which are inside a circle tangent to the frame of the $n \times n$ square, and the "cross" shape contains a horizontal and a vertical line that meet in the center of the square, both with a single-pixel width.

The images below show an example of how the different elements should look with a size of $9 \times 9$.
![Structuring_elements_showcase](./images/Structuring_elements_showcase.png "Structuring elements")

<div class=" alert alert-info">

<b>Note:</b> Because all the proposed structuring elements are all symmetric and we use $n$ odd, it is not necessary to reflect the structuring element as we would do in a generic implementation. However, you can still do it if you want.
</div>

## <a class="anchor"></a> 1.A. Square structuring element
[Back to index](#-Index)

The next cell provides the example function `square(n)`, which returns a square of size $n \times n$ filled with $1$s.

In [6]:
%use javascript
// function that takes as input an integer n and returns an n x n image of '1's
function square(n){ 
    // declare the output image
    var output = new Image(n, n);
    // iterate through each pixel
    for(var x = 0; x < n; x++){  
        for(var y = 0; y < n; y++){
            // assign pixel value at location (x,y) to '1'
            output.setPixel(x, y, 1);
        }
    }
    return output;
}

The next cell runs the function `square(n)` and stores the result in the variable `strel_square` which is converted to Python in order to display it in another cell. Feel free to experiment with the size passed to the function and observe the results.
<div class=" alert alert-info">
    
<b>Remember:</b> The method `.toArray()` is needed to convert the variable to Python.
    
</div>

In [7]:
%use javascript
%put strel_square
// runs the function you implemented above and converts the Image object to an array
// feel free to change the size passed to this function and observe the result
var strel_square = square(9).toArray();

Run the next cell to use Python to display the structuring element as an image.

<div class = 'alert alert-info'>
    
**Note:** Throughout the lab, we will reuse the following general cell structure: 
1. Function/Code, 
2. Running the function, 
3. Display/evaluate the results

We will not give a detailed description every time and we will let you do more and more on your own as we progress.
</div>

In [8]:
%use sos
# Display the binary image with a title and numerated pixel grid
plt.close('all')
disp_square = viewer(np.array(strel_square), title=f'Square structuring element of size {np.shape(strel_square)}', 
                     clip_range = [0, 1], axis=True, pixel_grid=True, cmap='viridis')

Button(description='Show Widgets', style=ButtonStyle())

When working in Python, we can use the [OpenCV](https://docs.opencv.org/master/) library to generate structuring elements and perform image processing tasks.

To use OpenCV (called `cv2` in Python) we usually import it as `cv`, which has already been done in the first cell of the notebook. When using the morphological filters provided by OpenCV, the structuring element which should be passed to the functions is a `numpy` array. We can either define it ourselves using NumPy, for example, a $9 \times 9$ square as in the cell below

In [9]:
%use sos
strel_square_custom = np.ones((9,9))

or we can use [`cv.getStructuringElement()`](https://docs.opencv.org/trunk/d4/d86/group__imgproc__filter.html#gac342a1bb6eabf6f55c803b09268e36dc), which can generate rectangles ([`cv.MORPH_RECT`](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gac2db39b56866583a95a5680313c314ad)), ellipses ([`cv.MORPH_ELLIPSE`](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gac2db39b56866583a95a5680313c314ad)) and crosses ([`cv.MORPH_CROSS`](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gac2db39b56866583a95a5680313c314ad)). The first argument is the type of the structuring element and the second is the size. Run the next cell to get the same $9 \times 9$ square we defined above.

In [10]:
%use sos
strel_square_cv = cv.getStructuringElement(cv.MORPH_RECT, (9,9))

We can compare the three structuring elements (JS, NumPy, OpenCV). We can do this either visually by providing the `IPLabViewer` class with a list of images and titles, as in the cell below

<div class = 'alert alert-warning'>

**Note:** If you changed the size of the JS structuring element, change it back to $9$ for the following comparisons. 
</div>

In [11]:
%use sos

# Close existing figures to release memory
plt.close('all')
# Display the three structuring elemnts side by side to compare them
disp_square = viewer([np.array(strel_square), strel_square_custom, strel_square_cv], title=['JavaScript', 'Custom numpy array', 'OpenCV'], 
                     subplots=(1,3), clip_range = [0, 1], cmap='viridis', axis=True, pixel_grid=True)

Button(description='Show Widgets', style=ButtonStyle())

or numerically, to make sure all the implementations provide the exact same result by using the `assert` statement, as in the cell below.

<div class="alert alert-info">

**Note:** [`np.count_nonzero(arr)`](https://numpy.org/doc/stable/reference/generated/numpy.count_nonzero.html) returns the number of non-zero elements in `arr`.
</div>

In [12]:
%use sos
# Compare the JS version to the custom numpy array by substracting one from the other and then counting the number of non-zero pixels, which should be 0
assert np.count_nonzero(strel_square - strel_square_custom) == 0, 'The JS and NumPy version do not agree.'
# Compare the JS version to the OpenCV version in the same way
assert np.count_nonzero(strel_square - strel_square_cv) == 0, 'The JS and OpenCV version do not agree.'
print('Indeed, the three structuring elements are exactly the same.')

Indeed, the three structuring elements are exactly the same.


## <a class="anchor"></a> 1.B. Cross structuring element
[Back to index](#-Index)

Now it's your turn!

In the cell below, **for 1 point**, complete the code in JavaScript to implement the `cross(n)` function.

<div class=" alert alert-info">
    
<b>Hint:</b> In JS all variables are in floating point precision. If you need to, you can use the function `parseInt(x)` to get the largest integer smaller than a given $x\in\mathbb{R}$.
</div>

In [13]:
%use javascript

// function that takes as input an integer n and returns the cross structuring element of size (n x n)
function cross(n){ 
    // declare output image
    var output = new Image(n, n);
    
    // YOUR CODE HERE
    var c = Math.floor(n/2);
    for(var i = 0; i < n; ++i) {
        output.setPixel(c, i, 1);
        output.setPixel(i, c, 1);
    }
    return output;
}

In [14]:
%use javascript
%put strel_cross
// runs the function you implemented above and converts the Image object to an array
// feel free to change the size passed to this function and observe the result
var strel_cross = cross(9).toArray();

In [15]:
%use sos
# Display the binary image with a title and numbered pixel grid
plt.close('all')
disp_cross = viewer(np.array(strel_cross), title=f'Cross structuring element of size {np.shape(strel_cross)}', 
                    axis=True, pixel_grid=True, cmap='viridis', clip_range = [0, 1])

Button(description='Show Widgets', style=ButtonStyle())

In Python you can use OpenCV to generate a cross structuring element by passing `cv.MORPH_CROSS` to `getStructuringElement`. Run the cell below to create and display the OpenCV cross. You can also change the size of the structuring element and see the result. 

In [16]:
%use sos
# Generate the cross structuring element with OpenCV - feel free to play with the size passed to this function
strel_cross_cv = cv.getStructuringElement(cv.MORPH_CROSS, (9, 9))
# Display it
plt.close('all')
disp_cross_cv = viewer(strel_cross_cv, title=f'Python cross of size {np.shape(strel_cross_cv)}', 
                       axis=True, pixel_grid=True, cmap='viridis', clip_range = [0, 1])

Button(description='Show Widgets', style=ButtonStyle())

The following cell tests if the Python and JavaScript structuring elements are identical. To make sure you pass the test, verify that both are of the same size!

<div class = 'alert alert-danger'>

**Note:** Throughout this section, we will give you the freedom to choose the size of the JS and the OpenCV structuring element. However, if you do change it, make sure to change it back to the original for the following comparisons. This is true for every exercise. When you hand in your notebook, **all comparison cells should run without any error.**
</div>

In [17]:
%use javascript
%get strel_cross_cv
if(Image.arrayCompare(Image.shape(strel_cross), Image.shape(strel_cross_cv)) == false){
    throw new Error('The size of the two structuring elements is not the same:\nstrel_cross = (' + Image.shape(strel_cross) + '), strel_cross_cv = (' + Image.shape(strel_cross_cv) + ')');
}
if(Image.arrayCompare(strel_cross, strel_cross_cv) == false){
    throw new Error('The two structuring elements are not the same.');
}
// print a victory message
console.log("Yes! The crosses are identical.");

Yes! The crosses are identical.


## <a class="anchor"></a> 1.C. Disc structuring element 
[Back to index](#-Index)

In the cell below, **for 1 point**, complete the code in JavaScript to implement the `disc(n)` function.

In [18]:
%use javascript

// function that takes as input an integer n and returns the disc structuring element of size (n x n)
function disc(n){ 
    // Ddclare output image
    var output = new Image(n, n);
    
    // YOUR CODE HERE
    var r = Math.floor(n/2);
    var r2 = r*r;
    for(var i = 0; i < n; ++i) {
        for(var j = 0; j < n; ++j) {
            if((i-r)*(i-r)+(j-r)*(j-r) <= r2)
                output.setPixel(i, j, 1);
        }
    }
    
    return output;
}

In [19]:
%use javascript
%put strel_disc
// runs the function you implemented above and converts the Image object to an array
// feel free to change the size passed to this function and observe the result
var strel_disc = disc(9).toArray();

In [20]:
%use sos
# Display the binary image with a title and numerated pixel grid
plt.close('all')
disp_disc = viewer(np.array(strel_disc), title=f'Disc structuring element of size {np.shape(strel_disc)}', 
                   axis=True, pixel_grid=True, cmap='viridis', clip_range = [0, 1])

Button(description='Show Widgets', style=ButtonStyle())

Unfortunately, OpenCV doesn't provide a disc structuring element. However, it provides elliptical structuring elements by passing `cv.MORPH_ELLIPSE` to `getStructuringElement`. Usually, an ellipse which has the same height and width should be just a normal circle/disc, so lets see what happens if we generate such a disc using OpenCVs ellipse generator. Run the cell below to generate OpenCVs interpretation of a disc.

In [21]:
%use sos
# Generate an ellipse with height = width
strel_ellipse = cv.getStructuringElement(cv.MORPH_ELLIPSE, (9,9))
# Display the result
plt.close('all')
disp_ellipse = viewer(strel_ellipse, title='OpenCVs definition of a disc', pixel_grid=True, axis=True, cmap='viridis')

Button(description='Show Widgets', style=ButtonStyle())

<a name="disc_function"></a>
Don't worry, you didn't do anything wrong. As you can hopefully see, this isn't a circular disc, but still some kind of ellipse, even though the height and width are set to the same value. Either this is a bug in the implementation of `getStructuringElement()` or they simply have another interpretation of a circle (which is wrong). Either way this means we need to create our own function in Python to generate a disc structuring element. Luckily for you, we have already done this and will provide it to you in the following cell. 
<div class=" alert alert-info">
    
**Note:** Because it is very inefficient to use for loops in Python (and you should only do it if absolutely necessary!), this task was implemented using a [`lambda` function](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) and the NumPy method [`np.fromfunction`](https://numpy.org/doc/stable/reference/generated/numpy.fromfunction.html).
</div>

<div class=" alert alert-success">
    
**Note:** For the scope of this lab you do not need to understand the `lambda` function. However, we do recommend you to go through the documentation and to completely understand the following cell $-$ it will improve your programming skills.
</div>

In [22]:
%use sos
# Function that generates a disc structuring element in python
def disc(n):
    # Define the function of a circle as a lambda function
    circle_func = lambda i, j: ((i - n//2)**2 + (j - n//2)**2) <= (n//2)**2
    # Set all elements of the array that are inside the circle of diamater n to 1 - np.uint8 to match the type used by OpenCV for structuring elements
    output = np.fromfunction(circle_func, shape=(n,n)).astype(np.uint8)
    # Return the structuring element
    return output

In [23]:
%use sos
# Here we generate a disc structuring element with python by calling the function implemented above
strel_disc_python = disc(9)
# And display it
plt.close('all')
disp_disc = viewer(strel_disc_python, title=f'Python disc of size {np.shape(strel_disc_python)}', 
                   pixel_grid=True, axis=True, cmap='viridis')

Button(description='Show Widgets', style=ButtonStyle())

Now that we have a working function to compare your implementation to, lets do it! Run the cell below to test your implementation in JS.

In [24]:
%use javascript
%get strel_disc_python
// this cell tests if the two tructuring elements (JavaScript and Python) are identical, which they should be
if(Image.arrayCompare(Image.shape(strel_disc), Image.shape(strel_disc_python)) == false){
    throw new Error('The size of the two structuring elements is not the same:\nstrel_cross = (' + Image.shape(strel_disc) + '), strel_cross_cv = (' + Image.shape(strel_disc_python) + ')');
}
if(Image.arrayCompare(strel_disc, strel_disc_python) == false){
    throw new Error('The two structuring elements are not the same.');
}
// print a victory message
console.log("Well done! Your disc is better than that of OpenCV.");

Well done! Your disc is better than that of OpenCV.


# <a class="anchor"></a> 2. Debugging (1 point)
[Back to index](#-Index)

The provided function `erodeBug()`, which is given in the next cell, is supposed to perform an erosion operation on an image with a square structuring element of size $3 \times 3$. While the JavaScript syntax is correct, there are two bugs in the implementation of `erodeBug()`. Inspect the code below, fix the two bugs, and explore the cells below to run the function on the images `plate` and `butterfly` to see the result.
<div class=" alert alert-info">
    
<b>Remember:</b>
- `img.nx` and `img.ny` can be used to get the dimensions of `img`,
- `img.getNbh(x, y, h, w)` returns the neighbourhood of size $(h\times w)$ around the location $(x,y)$ of `img`,
- `Math.min(a,b)` calculates the minimum between $a$ and $b$.
</div>

In [126]:
%use javascript

// function that erodes a structure with a (3 x 3) square. Original contains two bugs.
function erodeBug(img){ 
    // the structuring element b should be a 3x3 square
    var output = new Image(img.shape()); // CREATE IMAGE COPY
    var b = square(3);
    // loop through every pixel of the image
    for(var x = 0; x < img.nx; ++x){ // CHANGE TYPO ny TO nx 
        for(var y = 0; y < img.ny; ++y) {             
            // extract the 3x3 neighbourhood around pixel (x,y)
            var neigh = img.getNbh(x, y, 3, 3);
            // initializing the minimum value to the largest number possible in JS
            var valmin = Number.MAX_VALUE;
            // loop through every pixel of the neighbourhood
            for(var k = 0; k < 3; ++k){  
                for(var l = 0; l < 3; ++l){
                    // check if the structuring element is either 'true' or '1' at the pixel location
                    if(b.getPixel(k, l) == true || b.getPixel(k, l) == 1) { 
                        // calculate new minimum value
                        valmin = Math.min(neigh.getPixel(k, l), valmin);
                    }
                }
            }
            // set the pixel at location (x,y) to the calculated minimum value            
            output.setPixel(x, y, valmin);
        }
    }    
    return output;
}

Run the next cell to apply the function `erodeBug()` to both images and put the result in Python.

In [127]:
%use javascript
%get plate
%get butterfly
%put eroded_plate
%put eroded_butterfly

// convert the images to Image objects
var plate_img = new Image(plate);
var butterfly_img = new Image(butterfly);
// run erodeBug for the plate image
var eroded_plate = erodeBug(plate_img).toArray()
// run erodeBug for the butterfly image
var eroded_butterfly = erodeBug(butterfly_img).toArray()

Run the next cell to see the result of your function erodeBug, as well as the difference between the original and your result.

In [128]:
%use sos
# Defining the images and their titles
images = [plate, np.array(eroded_plate), plate-np.array(eroded_plate),  butterfly, np.array(eroded_butterfly),butterfly-np.array(eroded_butterfly)]
titles = ['Original plate', 'Eroded plate', 'Difference between the two', 'Original butterfly', 'Eroded butterfly', 'Difference between the two']
# Close all previous figures to release memory
plt.close('all')
# Display the images with their titles (you can pass lists of images and titles as arguments)
disp_erodeBug = viewer(images, title=titles, subplots=(2,3))

Button(description='Show Widgets', style=ButtonStyle())

If you're unsure about the result, it's always a good idea to test a function on an input to which we know the output. For example, we know that if we erode a $3 \times 3$ square surrounded by zeros with a $3 \times 3$ square, the result should be a black image with one single white pixel in the center of the original square, right? So lets try it... Run the cells below to create the test image and apply the function `erodeBug` on it.

In [129]:
%use sos
%put square_image --to javascript
# To define the image with a square in the middle, we initialize a 9x9 image of zeros
square_image = np.zeros((5, 11))
# and insert the 3x3 quare of ones towards the right side
square_image[1:4, 7:10] = 1
# Let's see how it looks
img_square_image = viewer(square_image, title='3x3 square sorrounded by zeros', pixel_grid=True, axis=True, subplots=(1,1))

Button(description='Show Widgets', style=ButtonStyle())

In [130]:
%use javascript
%put eroded_square
// run the function erodeBug on the square image
var eroded_square = erodeBug(new Image(square_image)).toArray();

As mentioned above, the result should be a single white pixel located in the center of the white square, in an otherwise black image. If this isn't the case, there might still be a bug in the `erodeBug` function above.

In [131]:
%use sos
# Display the result of the eroded square
plt.close('all')
img_eroded_square = viewer(np.array(eroded_square), title='erodeBug() on square image', pixel_grid=True, axis=True, clip_range = [0, 1], subplots=(1,1))

Button(description='Show Widgets', style=ButtonStyle())

# <a class="anchor"></a> 3. Morphological filters (9 points)
[Back to index](#-Index)

In this part you are asked to implement the morphological filters given in the table below. Click on their names for a quick link to where you have to implement them, and to [Back to table](#-3.-Morphological-filters-(9-points)) to come back here.

| $\text{Filter}$ | $\text{Function}$ | $\text{Definition / Mathematical notation}$   |
|------------|---------------|----------------------------------------------------------|
| [Erosion](#-3.A.-Erosion)              (3.A) | `erosion()`   | $f \ominus b$                                            |
| [Dilation](#-3.B.-Dilation)            (3.B) | `dilation()`  | $f \oplus b$                                             |
| [Open](#-3.D.-Opening)                 (3.D) | `open()`      | $f \circ b = (f \ominus b) \oplus b$                     |
| [Close](#-3.E.-Closing)                (3.E) | `close()`     | $f \bullet b = (f \oplus b) \ominus b$                   |
| [Gradient](#-3.F.-Gradient-filter)     (3.F) | `gradient()`  | $\bigtriangledown (f, b) = (f \oplus b) - (f \ominus b)$ |
| [Top-hat](#-3.G.-Top-hat-filter)       (3.G) | `topHat()`    | $\mathrm{TH}(f, b) = f - (f \circ b)$                    |
| [Bottom-hat](#-3.H.-Bottom-hat-filter) (3.H) | `bottomHat()` | $\mathrm{BH}(f, b) = (f \bullet b) - f$                  |
| [Median](#-3.C.-Median-filter)         (3.C) | `median()`    | $\mathrm{MED}(f, b)$                                     |

Each function is worth **1 point**. You will start by implementing the most basic morphological filters: `erosion()`, `dilation()`, and `median()` in JavaScript and compare them to the OpenCV equivalents. The rest will be implemented in Python. The results of your implementation will be shown on the images `plate` and `butterfly`.

## <a class="anchor"></a> 3.A. Erosion
[Back to table](#-3.-Morphological-filters-(9-points)), [Back to index](#-Index)



In the cell below, **for 1 point**, complete the code in JavaScript to implement the `erosion()`.

Remember that in the course, given an image $f$ with support $\Omega_f$ and a structuring element $b$ with support $\Omega_b$, we define erosion as

$$
    (f \ominus b)[\mathbf{k}] = \min_{\mathbf{q}\in\Omega_b}\left\lbrace f\left[\mathbf{k} + \mathbf{q}\right] \mid (\mathbf{k}+\mathbf{q})\in\Omega_f \right\rbrace\,.
$$
 
As we mentioned before, because all the structuring elements are symmetric and $N$ is odd, you need not worry about reflecting them. See the note in [1. Structuring elements](#-1.-Structuring-elements-(2-points)).

<div class="alert alert-info">
    
**Hint:** You can use `Number.MAX_VALUE` and `Number.MIN_VALUE` to get the highest and lowest numbers possible in JavaScript.

</div>

In [132]:
%use javascript

// function that performs an erosion on the image 'img' using the structuring element 'b'
function erosion(img, b){
    // declaring the output image
    var output = new Image(img.shape());
    
    for(var x = 0; x < img.nx; ++x){ // CHANGE TYPO ny TO nx 
        for(var y = 0; y < img.ny; ++y) {             
            // extract the 3x3 neighbourhood around pixel (x,y)
            var neigh = img.getNbh(x, y, b.nx, b.ny);
            // initializing the minimum value to the largest number possible in JS
            var valmin = Number.MAX_VALUE;
            // loop through every pixel of the neighbourhood
            for(var k = 0; k < b.nx; ++k){  
                for(var l = 0; l < b.ny; ++l){
                    // check if the structuring element is either 'true' or '1' at the pixel location
                    if(b.getPixel(k, l) == true || b.getPixel(k, l) == 1) { 
                        // calculate new minimum value
                        valmin = Math.min(neigh.getPixel(k, l), valmin);
                    }
                }
            }
            // set the pixel at location (x,y) to the calculated minimum value            
            output.setPixel(x, y, valmin);
        }
    }    
    return output;    
}

// here we declare the structuring element
var b = square(5); // Feel free to change it to your liking (using the functions implemented in part 1) and observe the results.

Run the next cell to apply the `erosion()` function to the images `plate` and `butterfly`.

In [133]:
%use javascript
%put plate_erosion
%put butterfly_erosion

// running the operation and converting the images back to python
var plate_erosion = erosion(new Image(plate), b).toArray();
var butterfly_erosion = erosion(new Image(butterfly), b).toArray();

Run the next cell to visualize the results.

In [134]:
%use sos
# Declare the lists of images and titles for the display
images = [plate, butterfly, np.array(plate_erosion), np.array(butterfly_erosion)]
image_names = ['plate', 'butterfly', 'plate eroded', 'butterfly eroded']

# Display all 4 images
plt.close('all')
erosion_results = viewer(images, title = image_names, subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

In Python, we can use `cv.erode(img, b, borderType=cv.BORDER_REFLECT)` to erode an image `img` with a structuring element `b` using mirror padding (`cv.BORDER_REFLECT`). Run the next cells to compare your erosion to the OpenCV erosion.

<div class="alert alert-info">
    
**Note:** We use mirror padding here specifically because the `getNbh` method used in JavaScript applies this padding by default. This way we can correctly compare the two results.

In [135]:
%use sos
# Lets erode the plate image with OpenCV to see if the results are the same
# Define the structuring element to use - feel free to change it, but it should be the same used in the JavaScript function for the comparison to work
b = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
# Erode plate
plate_erosion_cv = cv.erode(plate, b, borderType=cv.BORDER_REFLECT)

# Compare the two versions visually
plt.close('all')
erosion_comp = viewer([np.array(plate_erosion), plate_erosion_cv], title=['JS eroded plate', 'OpenCV eroded plate'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

In [136]:
%use sos
# And numerically
assert np.count_nonzero(plate_erosion - plate_erosion_cv) == 0, 'Sorry, but the eroded images are not identical. Make sure you used the same structuring elements for both functions.'
print('Nice! Your erosion function works perfectly.')

Nice! Your erosion function works perfectly.


## <a class="anchor"></a> 3.B. Dilation
[Back to table](#-3.-Morphological-filters-(9-points)), [Back to index](#-Index)

In the cell below, **for 1 point**, complete the code in JavaScript to implement the `dilation()`.

Remember that in the course, given an image $f$ with support $\Omega_f$ and a structuring element $b$ with support $\Omega_b$, we define dilation as

$$
    (f \oplus b)[\mathbf{k}] = \max_{\mathbf{q}\in\Omega_b}\left\lbrace f\left[\mathbf{k} - \mathbf{q}\right] \mid (\mathbf{k}-\mathbf{q})\in\Omega_f \right\rbrace\,.
$$

As we mentioned before, because all the structuring elements are symmetric and $N$ is odd, you need not worry about reflecting them. See the note in [1. Structuring elements](#-1.-Structuring-elements-(2-points)).

In [137]:
%use javascript

// function that performs a dilation on the image 'img' using the structuring element 'b'
function dilation(img, b) {
    // declaring the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
        for(var x = 0; x < img.nx; ++x){ // CHANGE TYPO ny TO nx 
        for(var y = 0; y < img.ny; ++y) {             
            // extract the 3x3 neighbourhood around pixel (x,y)
            var neigh = img.getNbh(x, y, b.nx, b.ny);
            // initializing the maximum value to the lowest number
            var valmax = 0;
            // loop through every pixel of the neighbourhood
            for(var k = 0; k < b.nx; ++k){  
                for(var l = 0; l < b.ny; ++l){
                    // check if the structuring element is either 'true' or '1' at the pixel location
                    if(b.getPixel(k, l) == true || b.getPixel(k, l) == 1) { 
                        // calculate new minimum value
                        valmax = Math.max(neigh.getPixel(k, l), valmax);
                    }
                }
            }
            // set the pixel at location (x,y) to the calculated minimum value            
            output.setPixel(x, y, valmax);
        }
    }    
    return output;
}

// Here we declare the structuring element
var b = square(5); // Feel free to change it to your liking (using the functions implemented in part 1) and observe the results.

In [138]:
%use javascript
%put plate_dilation
%put butterfly_dilation

// running the operation and converting the images back to python
var plate_dilation = dilation(new Image(plate), b).toArray();
var butterfly_dilation = dilation(new Image(butterfly), b).toArray();

In [139]:
%use sos
# Define the image and title lists for the visualization
images = [plate, butterfly, np.array(plate_dilation), np.array(butterfly_dilation)]
image_names = ['plate', 'butterfly', 'plate dilated', 'butterfly dilated']

# Display the results
plt.close('all')
dilation_results = viewer(images, title = image_names, subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

As we did for the erosion, in Python we can use `cv.dilate(img, b, borderType=cv.BORDER_REFLECT)` to dilate an image `img` with a structuring element `b` using mirror padding (`cv.BORDER_REFLECT`). Run the cells below to compare your implementation to the one of OpenCV.

In [140]:
%use sos
# Lets dilate the plate image using OpenCV
# As always, we first define the structuring element we want to use
b = cv.getStructuringElement(cv.MORPH_RECT, (5,5))

# Then we perform the dilation
plate_dilation_cv = cv.dilate(plate, b, borderType=cv.BORDER_REFLECT)

# And compare the two versions visually
plt.close('all')
dilation_comp = viewer([np.array(plate_dilation), plate_dilation_cv], title=['JS dilated plate', 'OpenCV dilated plate'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

In [141]:
%use sos
# And compare numerically
assert np.count_nonzero(plate_dilation - plate_dilation_cv) == 0, 'Sorry, but the dilated images are not identical. Make sure you used the same structuring elements for both functions.'
print("That's it! Your dilation function seems to be correct.")

That's it! Your dilation function seems to be correct.


## <a class="anchor"></a> 3.C. Median filter
[Back to table](#-3.-Morphological-filters-(9-points)), [Back to index](#-Index)

In the cell below, **for 1 point**, complete the code in JavaScript to implement the `median()` filter.

Remember that in the course, given an image $f$ with support $\Omega_f$ and a structuring element $b$ with support $\Omega_b$, we define the median filter as

$$
    \mathrm{MED}(f, b)[\mathbf{k}] = \mathrm{median}\left( \left\lbrace f\left[\mathbf{k} - \mathbf{q}\right] \mid \mathbf{q}\in\Omega_b, (\mathbf{k}-\mathbf{q})\in\Omega_f \right\rbrace \right)\,.
$$



Here, $\mathrm{median}(\cdot)$ is a function that acts on a set of numbers, as usual in statistics (see, e.g., [here](https://en.wikipedia.org/wiki/Median#Finite_data_set_of_numbers)). These numbers are specified by the mathematical expression inside $\lbrace\cdot\rbrace$. 

As we mentioned before, because all the structuring elements are symmetric and $N$ is odd, you need not worry about reflecting them. See the note in [1. Structuring elements](#-1.-Structuring-elements-(2-points)).

<div class=" alert alert-info">
    
**Hint:** You can use `nbh.sort(b)` to get a sorted (low to high) 1D `Image` object of the pixels in `nbh` that are under the `True` values of the structuring element `b`. The sorted `Image` object consists of just one row, so the length of the sorted numbers is given by `sorted.nx`. Example: use `sorted.getPixel(2, 0)` to extract the $3^\text{rd}$ smallest value.
</div>

In [142]:
%use javascript

// function that performs a median on the image 'img' using the structuring element 'b'
function median(img, b) {
    // declaring the output image
    var output = new Image(img.shape());
    // YOUR CODE HERE
    for(var x = 0; x < img.nx; ++x){ // CHANGE TYPO ny TO nx 
        for(var y = 0; y < img.ny; ++y) {             
            // extract the 3x3 neighbourhood around pixel (x,y)
            var neigh = img.getNbh(x, y, b.nx, b.ny);
            var sorted = neigh.sort(b);
            // set the pixel at location (x,y) to the calculated minimum value            
            output.setPixel(x, y, sorted.getPixel(Math.floor(sorted.nx/2),0));
        }
    }    
    return output;
}

// here we declare the structuring element
var b = square(5); // Feel free to change it to your liking (using the functions implemented in part 1) and observe the results.

In [143]:
%use javascript
%put plate_median
%put butterfly_median

// running the operation and converting the images back to python
var plate_median = median(new Image(plate), b).toArray();
var butterfly_median = median(new Image(butterfly), b).toArray();

In [144]:
%use sos
# Define the lists of names and images for visualization
images = [plate, butterfly, np.array(plate_median), np.array(butterfly_median)]
image_names = ['plate', 'butterfly', 'plate median', 'butterfly median']

# Display the results
plt.close('all')
median_results = viewer(images, title = image_names, subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

To apply a median filter to an image `img` with an $n \times n$ **square** structuring element using OpenCV, we can use `cv.medianBlur(img, n)`. Run the cell below to compare your median to the OpenCV median.

<div class="alert alert-info">
    
**Note:**  In OpenCV, the median filter can only be applied with a square structuring element, and uses "repeat padding" at the border, i.e., it repeats the last pixel.

</div>

In [145]:
%use sos
# Lets apply the median to the plate image
# Set the size of the square structuring element
n = 5
# Run the operation
plate_median_cv = cv.medianBlur(plate, n)
# Compare the two versions visually
plt.close('all')
median_comp = viewer([np.array(plate_median), plate_median_cv], title=['JS median filtered plate', 'OpenCV median filtered plate'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

<div class="alert alert-info">

**Note:** Because the OpenCV median uses "repeat padding" instead of "mirror padding" as we do, we should not compare the border region of the images. To be sure, we leave the $\frac{n}{2} + 1$ outer-most rows and columns out of the comparison, where $n$ is the size of the square structuring element.
</div>

In [146]:
%use sos
# And numerically
assert np.count_nonzero(np.array(plate_median)[n//2+1:np.shape(plate_median)[0]-n//2, n//2+1:np.shape(plate_median)[1]-n//2] \
                        - plate_median_cv[n//2+1:plate_median_cv.shape[0]-n//2, n//2+1:plate_median_cv.shape[1]-n//2]) == 0, \
                        'Sorry, but your median filter still needs some work.'
print("Very good! Your median filter seems to be correct.")

Very good! Your median filter seems to be correct.


Now lets take a moment to look at the [morphological filters' table](#-3.-Morphological-filters-(9-points)), given in the beginning of Part 2. There, you will see that all the morphological filters that we have not implemented yet are simple combinations of some of those that we have implemented, namely `dilation` and `erosion`. Since the idea of this lab is for you to understand how morphological filters work and not to write down an unnecessary amount of `for` loops in JavaScript, we will now switch to only using Python and OpenCV for the rest of the lab.

## <a class="anchor"></a> 3.D. Opening
[Back to table](#-3.-Morphological-filters-(9-points)), [Back to index](#-Index)

In the cell below, **for 1 point**, implement the `open()` function **using only the `cv.dilate` and `cv.erode` functions presented above**.

<div class=" alert alert-warning">

**Beware:** Be consistent, make sure you use `cv.BORDER_REFLECT` as the border type for all functions.
</div>

In [147]:
%use sos

# Function that performs an opening on the image 'img' using the structuring element 'b'
def open(img, b):
    # Declaring the output image
    AoB = np.zeros(img.shape)
    
    # YOUR CODE HERE
    AoB = cv.dilate(cv.erode(img, b, borderType=cv.BORDER_REFLECT), b, borderType=cv.BORDER_REFLECT)
    return AoB

# Here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, (5,5)); # Feel free to change it to your liking (using the functions presented in part 1) and observe the results.

# Directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_open     = open(plate,     b)
butterfly_open = open(butterfly, b)

# and also display the result
# Define the lists of images and titles
images      = [plate, butterfly, plate_open, butterfly_open]
image_names = ['plate', 'butterfly', 'plate opened', 'butterfly opened']
# Visualize them
plt.close('all')
open_results = viewer(images, title = image_names, subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

Now lets compare your `open` function to the one that OpenCV provides. Using OpenCV, you can perform the opening of an image `img` with a structuring lement `b` using `cv.morphologyEx(img, cv.MORPH_OPEN, b, borderType=cv.BORDER_REFLECT)`. Again we use `borderType=cv.BORDER_REFLECT` for consistency. Run the next cell to compare the functions.

In [148]:
%use sos
# Perform the opening on plate with OpenCV using the same structuring element b
plate_open_cv = cv.morphologyEx(plate, cv.MORPH_OPEN, b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
assert np.count_nonzero(plate_open - plate_open_cv) == 0, 'Sorry, your opening is not quite right.'
print("Great! Your opening seems to be correct.")

Great! Your opening seems to be correct.


## <a class="anchor"></a> 3.E. Closing
[Back to table](#-3.-Morphological-filters-(9-points)), [Back to index](#-Index)

In the cell below, **for 1 point**, implement the `close()` function **using only the `cv.dilate` and `cv.erode` functions presented above**.

In [149]:
%use sos

# Function that performs a closing on the image 'img' using the structuring element 'b'
def close(img, b):
    # Declaring the output image
    AcB = np.zeros(img.shape)
    
    # YOUR CODE HERE
    AcB = cv.erode(cv.dilate(img, b, borderType=cv.BORDER_REFLECT), b, borderType=cv.BORDER_REFLECT)
    return AcB;

# Here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, (5,5)); # Feel free to change it to your liking (using the functions discussed and implemented in part 1) and observe the results.

# Directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_close     = close(plate,     b)
butterfly_close = close(butterfly, b)

# and also display the result
# Define the lists of images and titles
images = [plate, butterfly, plate_close, butterfly_close]
image_names = ['plate', 'butterfly', 'plate closed', 'butterfly closed']
# Visualize them
plt.close('all')
close_results = viewer(images, title = image_names, subplots = (2,2))

Button(description='Show Widgets', style=ButtonStyle())

To perform the closing operation with OpenCV, we use `cv.MORPH_CLOSE` instead of `cv.MORPH_OPEN` in the `cv.morphologyEx` function presented above. Run the cell below to compare the functions.

In [150]:
%use sos
# Perform the closing on plate with OpenCV using the same structuring element b
plate_close_cv = cv.morphologyEx(plate, cv.MORPH_CLOSE, b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
assert np.count_nonzero(plate_close - plate_close_cv) == 0, 'Sorry, your closing is not quite right.'
print("Great! Your closing seems to be correct.")

Great! Your closing seems to be correct.


## <a class="anchor"></a> 3.F. Gradient filter
[Back to table](#-3.-Morphological-filters-(9-points)), [Back to index](#-Index)

In the cell below, **for 1 point**, implement the `gradient()` function **using only the `cv.dilate` and `cv.erode` functions presented above** and basic arithmetics (`+`,`-`,`*`, or `/`).
<div class=" alert alert-info">

**Hint:** Remember that in Python you can add/subtract whole images simply by using the $+/-$ operators, without the need to iterate with `for` loops.
</div>

In [151]:
%use sos

# function that performs a gradient on the image 'img' using the structuring element 'b'
def gradient(img, b):
    # declaring the output image
    grad = np.zeros(img.shape)
    
    # YOUR CODE HERE
    grad = cv.dilate(img, b, borderType=cv.BORDER_REFLECT) - cv.erode(img, b, borderType=cv.BORDER_REFLECT)
    return grad

# here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, (5,5)); # Feel free to change it to your liking (using the functions discussed and implemented in part 1) and observe the results.

# directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_gradient     = gradient(plate,     b)
butterfly_gradient = gradient(butterfly, b)

# and also display the result
# define the lists of images and titles
images = [plate, butterfly, plate_gradient, butterfly_gradient]
image_names = ['plate', 'butterfly', 'plate gradient', 'butterfly gradient']
# visualize them
plt.close('all')
gradient_results = viewer(images, title = image_names, subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

In OpenCV, the identifier `cv.MORPH_GRADIENT` can be used to calculate the gradient with the `cv.morphologyEx` function presented above. Run the next cell to check your function.

In [152]:
%use sos
# Perform the closing on plate with OpenCV using the same structuring element b
plate_gradient_cv = cv.morphologyEx(plate, cv.MORPH_GRADIENT, b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
assert np.count_nonzero(plate_gradient - plate_gradient_cv) == 0, 'Sorry, your gradient is not quite right.'
print("Great! Your gradient seems to be correct.")

Great! Your gradient seems to be correct.


## <a class="anchor"></a> 3.G. Top-hat filter
[Back to table](#-3.-Morphological-filters-(9-points)), [Back to index](#-Index)

In the cell below, **for 1 point**, implement the `topHat()` function by **using only the OpenCV functions we have seen in Tasks [3.A.](#-3.A.-Erosion-operator) to [3.E.](#-3.E.-Close-operator)**.

In [153]:
%use sos

# function that performs a topHat on the image 'img' using the structuring element 'b'
def topHat(img, b):
    # declaring the output image
    tophat = np.zeros(img.shape)
    
    # YOUR CODE HERE
    tophat = img - cv.morphologyEx(img, cv.MORPH_OPEN, b, borderType=cv.BORDER_REFLECT)
    return tophat
    
# here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, (5, 5)); # Feel free to change it to your liking (using the functions discussed and implemented in part 1) and observe the results.

# directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_tophat     = topHat(plate,     b)
butterfly_tophat = topHat(butterfly, b)

# and also display the result
# define the lists of images and titles
images = [plate, butterfly, plate_tophat, butterfly_tophat]
image_names = ['plate', 'butterfly', 'plate tophat', 'butterfly tophat']
# visualize them
plt.close('all')
tophat_results = viewer(images, title = image_names, subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

To perform the topHat filter in OpenCV, one uses `cv.MORPH_TOPHAT`. Run the cell below to check your function.

In [154]:
%use sos
# Perform the closing on plate with OpenCV using the same structuring element b
plate_tophat_cv = cv.morphologyEx(plate, cv.MORPH_TOPHAT, b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
assert np.count_nonzero(plate_tophat - plate_tophat_cv) == 0, 'Sorry, your topHat filter is not quite right.'
print("Great! Your topHat filter seems to be correct.")

Great! Your topHat filter seems to be correct.


## <a class="anchor"></a> 3.H. Bottom-hat filter
[Back to table](#-3.-Morphological-filters-(9-points)), [Back to index](#-Index)

In the cell below, **for 1 point**, implement the `bottomHat()` function by **using only the OpenCV functions we have seen in Tasks [3.A.](#-3.A.-Erosion-operator) to [3.E.](#-3.E.-Close-operator)**.

In [155]:
%use sos

# function that performs a bottomHat on the image 'img' using the structuring element 'b'
def bottomHat(img, b):
    # declaring the output image
    bottomhat = np.zeros(img.shape)
    
    # YOUR CODE HERE
    bottomhat = cv.morphologyEx(img, cv.MORPH_CLOSE, b, borderType=cv.BORDER_REFLECT) - img
    return bottomhat

# here we declare the structuring element
b = cv.getStructuringElement(cv.MORPH_RECT, (5,5)); # Feel free to change it to your liking (using the functions discussed and implemented in part 1) and observe the results.

# directly run the function on the two images since we do not need to convert any variables between JavaScript and Python
plate_bottomhat     = bottomHat(plate,     b)
butterfly_bottomhat = bottomHat(butterfly, b)

# and also display the result
# define the lists of images and titles
images = [plate, butterfly, plate_bottomhat, butterfly_bottomhat]
image_names = ['plate', 'butterfly', 'plate bottomHat', 'butterfly bottomHat']
# visualize them
plt.close('all')
bottomhat_results = viewer(images, title = image_names, subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

In OpenCV we use `cv.MORPH_BLACKHAT` to perform a bottomHat filter. Run the next cell to check your function.

In [156]:
%use sos
# Perform the closing on plate with OpenCV using the same structuring element b
plate_bottomhat_cv = cv.morphologyEx(plate, cv.MORPH_BLACKHAT, b, borderType=cv.BORDER_REFLECT)
# Compare the two versions
assert np.count_nonzero(plate_bottomhat - plate_bottomhat_cv) == 0, 'Sorry, your bottomHat filter is not quite right.'
print("Great! Your bottomHat filter seems to be correct.")

Great! Your bottomHat filter seems to be correct.


## <a class="anchor"></a> 3.I. Understanding morphological filters
[Back to index](#-Index)

Which of following statements are correct? Here, $N\times$ Function(`name`, $y$) refers to composing a function $N$ times with the structuring element given by `name` with size $y\times y$.

1. The results of $3 \times$ Erosion(Square, 3) and Erosion(Square, 7) are the same.
2. The results of $3 \times$ Open(Cross, 5) and Open(Cross, 5) are the same.
3. The results of $3 \times$ Close(Disk, 5) and Close(Disk, 5) are the same.
4. The results of Top-Hat(Square, 3) and Bottom-Hat (Square, 3) are the same.

You can use the next cell to compare the different propositions by modifying the existing code and inserting your own.

<div class="alert alert-info">

**Note:** To get a disc structuring element you can use the [`disc(n)`](#-1.D.-Disc-Structuring-Element) function we provided in Part [1.C.](#-1.C.-Disc-structuring-element)
</div>

In [157]:
%use sos
# We use the butterfly image, feel free to use any other image
modified_1 = butterfly
modified_2 = butterfly

# Apply the morphological operators
b = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
modified_1 = cv.morphologyEx(modified_1, cv.MORPH_TOPHAT, b, borderType=cv.BORDER_REFLECT)

b = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
modified_2 = cv.morphologyEx(modified_2, cv.MORPH_BLACKHAT, b, borderType=cv.BORDER_REFLECT)

# YOUR CODE HERE

# Display the two modified images as well as their difference
images = [modified_1, modified_2, modified_2 - modified_1]
titles = ['Operation 1', 'Operation 2', 'Difference']

plt.close('all')
operation_comparison = viewer(images, title = titles, subplots=(1,3))

Button(description='Show Widgets', style=ButtonStyle())

<div class="alert alert-danger">

In the next cell assign `True` to the statements you think are correct and assign `False` to the statements you think are incorrect. The following cells are for you to check that your answers are valid.
</div>

In [158]:
%use sos
# Example: 'statement_0 = True' or 'statement_0 = False'
statement_1 = True
statement_2 = True
statement_3 = True
statement_4 = False

# YOUR CODE HERE

In [159]:
%use sos
# Perform sanity check on statement_1
assert statement_1 in [True, False], 'Assign either True or False to statement_1.'

In [160]:
%use sos
# Perform sanity check on statement_2
assert statement_2 in [True, False], 'Assign either True or False to statement_2.'

In [161]:
%use sos
# Perform sanity check on statement_3
assert statement_3 in [True, False], 'Assign either True or False to statement_3.'

In [162]:
%use sos
# Perform sanity check on statement_4
assert statement_4 in [True, False], 'Assign either True or False to statement_4.'

# <a class="anchor"></a> 4. Direct applications (2 points)
[Back to index](#-Index)

Now let's look at some applications of morphological filters.

Using **only one** of the morphological filters of [Part 3](#-3.-Morphological-operators-(9-points)) followed by a threshold operation, try to solve the following problems as best you can. Select an appropriate threshold value to get a binary image (0 / 255).
<div class="alert alert-info">

**Note:** A threshold operation on `img`, given a threshold `thresh`, can be performed using [`np.where`](https://numpy.org/doc/stable/reference/generated/numpy.where.html) like this: 

`img_binary = np.where(img > thresh, 255, 0).astype(np.uint8)`
</div>

<div class="alert alert-info">
    
**Note:** While your functions and OpenCV's functions are equivalent (e.g, `close(img, b)` is equivalent to `cv.morphologyEx(img, b, cv.MORPH_CLOSE)`), we recommend you to use OpenCV's functions, as these will be the functions you will use if you need to apply morphological operators in the future. 
</div>

## <a class="anchor"></a> 4.A. Disconnecting round objects
[Back to index](#-Index)

In the image `test_contact`, **for 1 point**, disconnect the roundish white objects while keeping at least 1 white pixel per roundish object.

![test-contact_showcase](./images/test-contact_showcase.png)

Run the next cell to visualize the image you will be working with in this exercise.

<div class="alert alert-info">

**Note:** Insert your code into the function `disconnect(img)`. In this exercise you don't need to explicitly threshold anything because the original image is already binary.

</div>

In [163]:
%use sos
img_vis = viewer(test_contact)

Button(description='Show Widgets', style=ButtonStyle())

In [164]:
%use sos

# Function that performs a single morphological filter with some structuring element of size n
def disconnect(img, n):
    # Initialize output
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE
    b = cv.getStructuringElement(cv.MORPH_RECT, (n,n))
    # b = cv.getStructuringElement(cv.MORPH_CROSS, (n,n))
    # b = disc(n)
    output = cv.erode(img, b, borderType=cv.BORDER_REFLECT)
    
    return output

Run the next cell to check that the result only consists of binary values **(object = 255, background = 0)** using [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

In [165]:
%use sos
# Check that the output is binary
check_bin = disconnect(test_contact, 25)
assert np.all( np.unique(check_bin)==[0,255] ), \
       'The output is not binary with values {0, 255}.'
print("Good, the output is binary with values {0,255}.")

Good, the output is binary with values {0,255}.


To make it easy for you to find the right size of the structuring element, we will add an interactive slider to the image display using the `IPLabViewer` class. Run the next cell to use the interactive widget.
<div class="alert alert-info">

**Note:** To use the slider, click the the button `Extra Widgets`. Then you can adjust the size of the structuring element with the slider and click the button `Disconnect` to apply the `disconnect()` function with the currently selected size on the original image.
</div>


In [192]:
%use sos
# Instantiate the size slider
size_slider = widgets.IntSlider(value=25, min=0, max=50, step=1, description='n')
# Instantiate the diconnect button
button = widgets.Button(description = 'Disconnect')

# Define the callback function of the button
def button_callback(image):
    # run the disconnect function on the image with the size indicated by the slider
    output = disconnect(image, n=size_slider.value)
    return output

# Display the image with the extra slider functionality
plt.close('all')
test_contact_display = viewer(test_contact, title = "Disconnect test_contact", new_widgets = [size_slider, button], callbacks = [button_callback], widgets = True)

In the following cell, assign `n` with the size of the structuring element you think works best for this task.

In [167]:
%use sos
# Assign the size of the structuring element
n = 14

# YOUR CODE HERE

In [168]:
%use sos
# Perform a sanity check on n
assert 0 < n < 50, 'Choose a size that makes sense.'

## <a class="anchor"></a> 4.B. Detecting horizontal lines
[Back to index](#-Index)

In the image `test_scratch`, **for 1 point**, detect the horizontal white lines of thickness 1 pixel.

![test-scratch_showcase](./images/test-scratch_showcase.png)

Run the next cell to visualize the image you will be working with in this exercise.

<div class="alert alert-info">

**Note:** Insert your code into the function `detect_hlines(img, n, threshold)`.

</div>

In [27]:
%use sos
img_vis = viewer(test_scratch)

Button(description='Show Widgets', style=ButtonStyle())

In [28]:
%use sos

# Function that performs a single morphological filter with some structuring element of size n followed by thresholding
def detect_hlines(img, n, threshold):
    # Initialize output
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE
    # b = cv.getStructuringElement(cv.MORPH_RECT, (n,n))
    b = cv.getStructuringElement(cv.MORPH_RECT, (n,n))
    # b = disc(n)
    #output = cv.erode(img, b, borderType=cv.BORDER_REFLECT)
    #output = cv.morphologyEx(img, cv.MORPH_CLOSE, b, borderType=cv.BORDER_REFLECT)
    output = cv.morphologyEx(img, cv.MORPH_TOPHAT, b, borderType=cv.BORDER_REFLECT)
    output = np.where(output > threshold, 255, 0).astype(np.uint8)
    return output

Run the next cell to check that the result only consists of binary values **(object = 255, background = 0)** using [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

In [29]:
%use sos
# You can enter any values that work for you for the size and threshold
n = 25
threshold = 125
# Check if the image consists of only one value
assert len(np.unique(detect_hlines(test_scratch, n, threshold))) != 1, \
       f"Your image consists of one value: {np.unique(detect_hlines(test_scratch, n, threshold))}. \
       Try changing the values for n and threshold. If this issue persists, you probably made a mistake in your code"

# Check if the image is binary
assert len(np.unique(detect_hlines(test_scratch, n, threshold))) == 2, \
       f"Your image is not binary, it still consists of {len(np.unique(detect_hlines(test_scratch, n, threshold)))} different values. \
       Check your thresholding operation."
    
# Check that the lower binary value is 0
assert np.unique(detect_hlines(test_scratch, n, threshold))[0] == 0, \
       f"The lower binary value should be 0, not {np.unique(detect_hlines(test_scratch, n, threshold))[0]}."

# Check that the upper binary value is 255
assert np.unique(detect_hlines(test_scratch, n, threshold))[1] == 255, \
       f"The upper binary value should be 255, not {np.unique(detect_hlines(test_scratch, n, threshold))[1]}."

# Print victory message
print(f'Perfect! Your output image consists of only two values: {{{np.unique(detect_hlines(test_scratch, n, threshold))[0]},{np.unique(detect_hlines(test_scratch, n, threshold))[1]}}}')

Perfect! Your output image consists of only two values: {0,255}


Again, we will make it easier for you to select the appropriate size and threshold values by adding an extra widget to the image display. Run the next cell to use the interactive widget.

In [30]:
%use sos
# Instantiate the size slider
size_slider = widgets.IntSlider(value=25, min=0, max=50, step=1, description='n')
# Instantiate the threshold slider
thresh_slider = widgets.IntSlider(value=125, min=0, max=255, step=1, description='threshold')
# Instantiate the diconnect button
button = widgets.Button(description = 'Detect Hlines')

# Define the callback function of the button
def button_callback(image):
    # Run the disconnect function on the image with the size indicated by the slider
    output = detect_hlines(image, n=size_slider.value, threshold=thresh_slider.value)
    return output

# Display the image with the extra slider functionality
plt.close('all')
test_scratch_display = viewer(test_scratch, title = "Detect horizontal lines", new_widgets = [size_slider, thresh_slider, button], 
                              callbacks = [button_callback], widgets = True)

In the following cell, assign `n` with the size of the structuring element and `threshold` with the threshold you think works best for this task.

In [173]:
%use sos
# Assign your values here
n = 3
threshold = 35

# YOUR CODE HERE

In [174]:
%use sos
# Perform a sanity check on n
assert 0 < n < 50, 'Choose a size that makes sense.'

In [175]:
%use sos
# Perform a sanity check on threshold
assert 0 < threshold < 255, 'Choose a threshold that makes sense.'

# <a class="anchor"></a> 5. Combining morphological filters (1 point)
[Back to index](#-Index)

**For 1 point**, using **one or several** of the morphological filters of [Part 3](#-3.-Morphological-operators-(9-points)) followed by a threshold operation, detect round white objects of diameter 20 +/- 4 pixels in the image `test_scratch`. Try to preserve their original shapes. Select an appropriate threshold value to get a binary image.
![test-scratch_round_showcase](./images/test-scratch_round_showcase.png)

<div class="alert alert-info">

**Note:** Insert your code into the function `detect_round(img)`. Because you can use as many operators as you like with different structuring elements of multiple sizes, it would be too complicated to generate interactive sliders for all of them. That means you need to insert the sizes of the structuring elements and the threshold value directly into the code and change them by hand to find the a combination that works for you. In this case it will be very beneficial to first think well about what you want to achieve.
</div>

In [59]:
%use sos

# Function that detects roundish white objects of diamater 20 +/- 4 pixels
def detect_round(img):
    # Initialize the output image
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE
#     b = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
    b=disc(5)
    output = cv.morphologyEx(img, cv.MORPH_CLOSE, b, borderType=cv.BORDER_REFLECT)
    #output = np.where(output > 20, 255, 0).astype(np.uint8)
#     b = cv.getStructuringElement(cv.MORPH_RECT, (25,25))
    b=disc(25)
    outputbig = cv.morphologyEx(output, cv.MORPH_OPEN, b, borderType=cv.BORDER_REFLECT)     
#     b = cv.getStructuringElement(cv.MORPH_RECT, (15,15))
    b=disc(15)
    outputsmall = cv.morphologyEx(output, cv.MORPH_OPEN, b, borderType=cv.BORDER_REFLECT) 
    output = outputsmall - outputbig 
    b=disc(5) #cv.getStructuringElement(cv.MORPH_RECT, (5,5))
    output = cv.morphologyEx(output, cv.MORPH_OPEN, b, borderType=cv.BORDER_REFLECT)
    output = np.where(output > 20, 255, 0).astype(np.uint8)
    return output

# Run the function on the image test_scratch
test_scratch_modified = detect_round(test_scratch)

# Display the images
plt.close('all')
images = [test_scratch, test_scratch_modified]
titles = ['Original', 'Round objects detected']

test_round_viewer = viewer(images, title = titles, subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

Run the next few cells to test if the output has the required properties (background = 0, foreground = 255).

In [32]:
%use sos
# Check if the image consists of only one value
assert len(np.unique(detect_round(test_scratch))) != 1, \
       f"Your image consists of one value: {np.unique(detect_round(test_scratch))}. You probably made a mistake in your code"
# Check if the image is binary
assert len(np.unique(detect_round(test_scratch))) == 2, \
       f"Your image is not binary, it still consists of {len(np.unique(detect_round(test_scratch)))} different values. Check your thresholding operation."
# Print victory message
print(f'Good, the image is binary.')

Good, the image is binary.


In [33]:
%use sos
# Check that the lower binary value is 0
assert np.unique(detect_round(test_scratch))[0] == 0, f"The lower binary value should be 0, not {np.unique(detect_round(test_scratch))[0]}."
# Print victory message
print(f'The lower binary value is correct.')

The lower binary value is correct.


In [179]:
%use sos
# Check that the upper binary value is 255
assert np.unique(detect_round(test_scratch))[1] == 255, f"The upper binary value should be 255, not {np.unique(detect_round(test_scratch))[1]}."
# Print victory message
print(f'The upper binary value is correct.')

The upper binary value is correct.


# <a class="anchor"></a> 6. Lantuéjoul's skeleton (2 points)
[Back to index](#-Index)

In this part we're going to implement a 2D skeletonizing algorithm. This process is commonly used in handwritten text recognition, fingerprint validation and [raster-to-vector](https://en.wikipedia.org/wiki/Image_tracing) conversion. 

## <a class="anchor"></a> 6.A. Classic Lantuéjoul's algorithm
[Back to index](#-Index)

Lantuéjoul's algorithm is an iterative erosion procedure that gives an approximation of the skeleton of an object. The input is a binary image (object = 255, background = 0). The output is also a binary image (skeleton = 255, background = 0).

![skeletonize_showcase](./images/skeletonize_showcase.png)

The algorithm makes $N$ successive erosions $e_n$ of the image until the objects are completely eroded (stop condition), using a $3\times 3$ cross as a structuring element. As a consequence, the number $N$ of iterations is variable and depends of the size of the objects to erode. In Python, this can be implemented using a while loop.
The skeleton is the union of $N$ partial skeletons $s_n$:

$$\mathrm{skel} =\bigcup_{n \in \lbrace1,2,\dots,N\rbrace} s_{n} = \bigcup_{n \in \lbrace1,2,\dots,N\rbrace}[ e_n - (e_n \circ b) ]\,.$$

Each partial skeleton $s_n$ is obtained by performing a Top-hat operation on an eroded image $e_n$ using a $3 \times 3$ square structuring element $b$.

### <a class="anchor"></a> 6.A.a. Implementing skeletonize
[Back to index](#-Index)

**For 1 point**, complete the function `skeletonize` that returns the skeleton and that writes $N$ in the console (use `print(N)`). Test your code on the image `test_skeleton`.

<div class="alert alert-info">

**Hint:** You can use [`np.count_nonzero(img)`](https://numpy.org/doc/stable/reference/generated/numpy.count_nonzero.html) to get the number of non-zero pixels in `img`.

**Hint:** The correct way to _add_ two **binary** images (to get the union above) is to use [`cv.bitwise_or(img_1, img_2)`](https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#bitwise-or).

</div>
<div class="alert alert-warning">

**Beware:** If you don't set the correct stopping condition in the while loop, it can run forever and block the execution of all other code. The cell should generate its output in a few seconds, otherwise it is likely that you created an infinite loop. If this happens you can click on `Kernel` in the toolbar on top and select `Interrupt` to stop the infinite loop. After that you can adjust your code and rerun the cell.

</div>

In [40]:
%use sos
# Load the test-skeleton image
test_img = plt.imread('images/test-skeleton.tif')

# Function that takes as input a binary image and returns its skeleton
def skeletonize(img):
    # Defining the output image (an array of zeros with the same shape as the input image of type 'uint8')
    output = np.zeros(img.shape, np.uint8)
    
    # YOUR CODE HERE
    N = 0
    b = cv.getStructuringElement(cv.MORPH_CROSS, (3,3))
    s = cv.getStructuringElement(cv.MORPH_RECT, (3,3))

#     temp = cv.morphologyEx(img, cv.MORPH_TOPHAT, b, borderType=cv.BORDER_REFLECT)
    while np.count_nonzero(img):
        temp = cv.morphologyEx(img, cv.MORPH_TOPHAT, s, borderType=cv.BORDER_REFLECT)
        img = cv.erode(img, b, borderType=cv.BORDER_REFLECT)
        output = cv.bitwise_or(output, temp)
        N = N+1
    print(N)
    
    return output
    
# Run the function on the test image
test_skeleton = skeletonize(test_img)
    
# Define the lists of images and names
images = [test_img, test_skeleton]
titles = ['Original', 'Skeleton']

# Display the images
plt.close('all')
skeletonize_viewer = viewer(images, title = titles, subplots=(1,2))

20


Button(description='Show Widgets', style=ButtonStyle())

Run the cell below to verify that the output of your function is binary.

In [41]:
%use sos
# Check that the output is binary
assert len(np.unique(test_skeleton)) == 2 and np.max(np.unique(test_skeleton)) == 255 and np.min(np.unique(test_skeleton)) == 0, 'The output is not binary with values {0, 255}.'

### <a class="anchor"></a> 6.A.b. Testing skeletonize
[Back to index](#-Index)

Apply your algorithm on the hands image located at _images/hands.tif_ and display the result.
<div class="alert alert-info">

**Hint:** You will first need to load the image from memory into a variable (**name the variable `hands`**, you will need it later!), then apply the function and finally display the image using the `viewer`.
</div>

In [42]:
%use sos

# YOUR CODE HERE
hands = plt.imread('images/hands.tif')
skeletonize_viewer = viewer(skeletonize(hands), title = titles, subplots=(1,2))

39


Button(description='Show Widgets', style=ButtonStyle())

In the next cell, assign to the variable `N` the number of erosions it took to generate the skeleton of the `hands` image.

In [43]:
%use sos
# Number of erosions to skeletonize the image
N = 39

# YOUR CODE HERE

In [44]:
%use sos
# Perform sanity check on n
assert 0 < N < 50, 'The selected number of erosions is most likely not correct.'

## <a class="anchor"></a> 6.B. Pruning and post-processing
[Back to index](#-Index)

As you can observe in the `hands` image, the algorithm creates undesired small branches in the skeleton. When the objects have a constant thickness, it is possible to prune the skeleton by constructing a skeleton as the union of the $M$th to $N$th partial skeletons, with $1 \leq M \leq N$, instead of the $N$th first partial skeletons. This should remove some of the unwanted branches.

**For 1 point**, implement this new method and test in the tasks below.

### <a class="anchor"></a> 6.B.a. Implementing pruning
[Back to index](#-Index)

Program the method `skeletonize_and_prune(img, M)` that returns the pruned skeleton, with $M$ a parameter of the function.

In [52]:
%use sos

# Function that takes as input a binary image as well as an integer m and returns its skeleton
# composed of the union from the mth to the last skeleton
def skeletonize_and_prune(img, M):
    # Defining the output image
    output = np.zeros(img.shape, np.uint8)
    
    # YOUR CODE HERE
#     if M == 0: return img
    N = 0
    b = cv.getStructuringElement(cv.MORPH_CROSS, (3,3))
    s = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
    prune = None
    while np.count_nonzero(img):
        img = cv.erode(img, b, borderType=cv.BORDER_REFLECT)
        N = N+1
        if N >= M:
            temp = cv.morphologyEx(img, cv.MORPH_TOPHAT, s, borderType=cv.BORDER_REFLECT)
            output = cv.bitwise_or(output, temp)
        
    #print(N)
    
    return output

Run the cell below to verify that the output of your function is binary.

In [53]:
%use sos
# Check that the output is binary
check_bin = skeletonize_and_prune(test_img, 0)
assert len(np.unique(check_bin)) == 2 and np.max(np.unique(check_bin)) == 255 and np.min(np.unique(check_bin)) == 0, \
       'The output is not binary with values [0, 255].'

Test your code on the image `hands` and play with the parameter $M$ (by adjusting the slider in the extra widget created) to see the difference in the skeletons. Run the cell below to launch the interactive widget and test your function. Feel free to try on other images too.

<div class="alert alert-info">

**Hint:** Note that pruning is only possible for $M \geq 1$, because a value of $M=0$ represents the original image. Use this fact to test that the result for $M = 0$ is the same as without pruning.

</div>

In [54]:
%use sos
# Instantiate the size slider
m_slider = widgets.IntSlider(value=25, min=0, max=50, step=1, description='M')
# Instantiate the diconnect button
button = widgets.Button(description = 'Skeletonize and Prune')

# Define the callback function of the button
def button_callback(image):
    # Run the disconnect function on the image with the size indicated by the slider
    output = skeletonize_and_prune(image, M=m_slider.value)
    return output

# Display the image with the extra slider functionality
plt.close('all')
skeletonize_prune_display = viewer(hands, title = "Skeletonize and prune test", new_widgets = [m_slider, button], 
                                   callbacks = [button_callback], widgets = True)

### <a class="anchor"></a> 6.B.b. Testing skeletonize and prune
[Back to index](#-Index)

Apply your algorithm on the image `b_letter`, choosing the appropriate $M$ to best capture the shape of the letter B. Run the next cell to launch the interactive widget.

In [55]:
%use sos

# Load the test image
b_letter = plt.imread('images/b-letter.tif')

# Instantiate the size slider
m_slider = widgets.IntSlider(value=25, min=0, max=50, step=1, description='M')
# Instantiate the diconnect button
button = widgets.Button(description = 'Skeletonize and Prune')

# Define the callback function of the button
def button_callback(image):
    # Run the disconnect function on the image with the size indicated by the slider
    output = skeletonize_and_prune(image, M=m_slider.value)
    return output

# Display the image with the extra slider functionality
plt.close('all')
skeletonize_prune_b_display = viewer(b_letter, title = "Skeletonize and prune B", new_widgets = [m_slider, button], callbacks = [button_callback], widgets = True)

In the next cell, assign to the variable `M` the value for $M$ you think works best to capture the shape of the letter $B$, while removing as many of the undesired small branches as possible.

In [189]:
%use sos
# Best value for m
M = 10

# YOUR CODE HERE

In [190]:
%use sos
# Perform sanity check on m
assert 0 < M < 50, 'The value for m is most likely not correct.'

# <a class="anchor"></a> 7. Cartoonize your picture!
[Back to index](#-Index)

Choose a natural picture of your choice from the internet or from your own collection (it shouldn't be too large, otherwise the operations will take a long time). If you're too busy to search for an image yourself you can also use the _natural_image.jpg_ provided in the 'images' folder. Using a combination of morphological operators, arithmetic operators, inversion, and threshold operations, give a cartoon effect to your picture! You can try to make it look like a painting or to give it a distortion effect, look at the possibilities! 
<div class="alert alert-info">

**Note:** You can invert an image using [`cv.bitwise_not(img)`](https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#bitwise-not)

</div>

In [56]:
%use sos
orig = plt.imread('images/natural_image.jpg')
img = None

# YOUR CODE HERE
# img = cv.medianBlur(orig, 9)
b = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
# img = cv.bitwise_not(img, orig)
# img = cv.morphologyEx(img, cv.MORPH_BLACKHAT, b, borderType=cv.BORDER_REFLECT)
# img = cv.skeletonize()
gray = cv.cvtColor(orig, cv.COLOR_BGR2GRAY)
edges = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 9, 5)
img = cv.bitwise_not(edges)
img = cv.morphologyEx(edges, cv.MORPH_CLOSE, b, borderType=cv.BORDER_REFLECT)
img = cv.erode(img, b, borderType=cv.BORDER_REFLECT)
# img = cv.bitwise_not(img)

cartoon_viewer = viewer([orig, img], title = ['Original', 'Cartoonized'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

<p><b>Congratulations on finishing Lab 3!</b></p>
<p>
    
Make sure to save your notebook (keep a copy on your personal computer for reference) and upload it to <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a> (name it *SCIPER_Morphology.ipynb*). Also, make sure to run it one more time and check that it runs without errors!
</p>

<div class="alert alert-danger">
<h4>Feedback</h4>
    <p style="margin:4px;">
    This is the last lab of the first edition of the image-processing laboratories using Jupyter Notebooks running on Noto. Do not leave before giving us your <a href="https://moodle.epfl.ch/mod/feedback/view.php?id=1123106">feedback here!</a></p>
</div>